# Loading data

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
! unzip ml-1m.zip -d .

--2016-10-20 22:07:56--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org... 128.101.34.146
Connecting to files.grouplens.org|128.101.34.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: 'ml-1m.zip'

ml-1m.zip           100%[=====================>]   5.64M  1.17MB/s   in 4.9s   

2016-10-20 22:08:01 (1.15 MB/s) - 'ml-1m.zip' saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ./ml-1m/
  inflating: ./ml-1m/movies.dat      
  inflating: ./ml-1m/ratings.dat     
  inflating: ./ml-1m/README          
  inflating: ./ml-1m/users.dat       


In [2]:
import pandas as pd
import numpy as np

In [3]:
ratings = (pd.read_csv('./ml-1m/ratings.dat', engine='python', sep='::', names=['user', 'item', 'rating', 'timestamp'])
    .assign(timestamp=lambda df:pd.to_datetime(df.timestamp * 1000000000))
          )

movies = (pd.read_csv('./ml-1m/movies.dat', engine='python', sep='::', names=['item', 'title', 'genres'])
          .assign(genres=lambda df:df.genres.str.split('|').values)
          .set_index('item', drop=False))

# See http://files.grouplens.org/datasets/movielens/ml-1m-README.txt for more details
users = pd.read_csv('./ml-1m/users.dat', engine='python', sep='::', 
                    names=['user', 'gender', 'age', 'occupation', 'zipcode'])\
    .set_index('user', drop=False)

In [4]:
movies.head()

,item,title,genres
item,,,
1,1,Toy Story (1995),"[Animation, Children's, Comedy]"
2,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
3,3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,4,Waiting to Exhale (1995),"[Comedy, Drama]"
5,5,Father of the Bride Part II (1995),[Comedy]


In [5]:
users.head()

,user,gender,age,occupation,zipcode
user,,,,,
1,1,F,1,10,48067
2,2,M,56,16,70072
3,3,M,25,15,55117
4,4,M,45,7,02460
5,5,M,25,20,55455


In [6]:
print(ratings.shape)
ratings.head()

(1000209, 4)


,user,item,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


## Train/test split

 * Ideally time based split
 * For the sake of simplicity, let's just sample ratings uniformly (breaking the time machine rule)

In [8]:
test = ratings.sample(n=100000, random_state=0)
train_ratings_mask = ~ratings.index.isin(test.index)
train = ratings.loc[train_ratings_mask]

test_user_items = test[['user', 'item']]

print(train.shape)
print(test.shape)

(900209, 4)
(100000, 4)


## Evaluation function and 1st baseline

In [10]:
def rmse(predicted_ratings, ground_truth_ratings=test):

    # predicted rating will be nan if no prediction => returning nan as a result if not all test ratings provided
    joined_ratings = pd.merge(
        ground_truth_ratings,
        predicted_ratings, 
        on=['user', 'item'], how='left', suffixes=['_ground_truth', '_predicted'])

    squared_errors = np.power(joined_ratings.rating_ground_truth - joined_ratings.rating_predicted, 2)
    
    return np.sqrt(np.average(squared_errors))

In [11]:
class AverageTrainingModel:
    
    def __init__(self, average_rating):
        self.average_rating = average_rating

    @classmethod
    def train_model(cls, train_ratings):
        """ Factory method
        """
        average_rating = train_ratings.rating.mean()
        return cls(average_rating)
    
    def average_ratings_array(self, user_items):
        return np.repeat(self.average_rating, user_items.shape[0])
    
    def predict(self, user_items):
        return user_items.assign(rating=self.average_ratings_array)

In [12]:
average_model = AverageTrainingModel.train_model(train)
average_ratings = average_model.predict(test_user_items)

print(rmse(predicted_ratings=average_ratings))
average_ratings.head()

1.11416005105


,user,item,rating
324271,1922,2094,3.581826
818637,4918,2808,3.581826
148677,957,1660,3.581826
778790,4653,914,3.581826
525489,3245,3324,3.581826


## User and item biases as a linear model



In [14]:
from scipy import sparse
from sklearn.linear_model import LinearRegression


def to_coo(row_indexes, col_indexes, values, shape=None, dtype=np.float64):
    return sparse.coo_matrix((values, (row_indexes, col_indexes)), shape=shape, dtype=dtype)

n_samples, _ = train.shape

user_features = to_coo(
    row_indexes=np.arange(n_samples), 
    col_indexes=train.user.values, 
    values=np.repeat(1, n_samples))

user_features

<900209x6041 sparse matrix of type '<class 'numpy.float64'>'
	with 900209 stored elements in COOrdinate format>

In [15]:
def to_indicator_features(user_items):
    n_samples, _ = user_items.shape
    user_features = to_coo(
        row_indexes=np.arange(n_samples), 
        col_indexes=user_items.user.values, 
        values=np.repeat(1, n_samples))

    item_features = to_coo(
        row_indexes=np.arange(n_samples), 
        col_indexes=user_items.item.values, 
        values=np.repeat(1, n_samples))

    return sparse.hstack([user_features, item_features])

indicator_features_matrix = to_indicator_features(user_items=train)
indicator_features_matrix

<900209x9994 sparse matrix of type '<class 'numpy.float64'>'
	with 1800418 stored elements in COOrdinate format>

In [16]:
m = LinearRegression(fit_intercept=True)
m.fit(X=indicator_features_matrix, y=train.rating)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
m.intercept_

3.5818259981848661

In [19]:
test_features = to_indicator_features(test_user_items)
predicted_ratings = m.predict(X=test_features)

user_item_bias_ratings = test_user_items.assign(rating=lambda _: predicted_ratings)

rmse(user_item_bias_ratings)

0.90726004853690523